In [ ]:
!pip install pandas rasterio netCDF4 rioxarray affine geopandas

import geopandas as gpd
import os
from datetime import timedelta
import requests
import json
import xml.etree.ElementTree as ET
import certifi
import pandas as pd
import numpy as np
from rasterio.windows import Window
import netCDF4 as nc
from rasterio.control import GroundControlPoint

# Use csv file to get the list of files to download
dates = pd.read_csv("Heat Wave Days.csv",usecols=["Date"])["Date"].to_list()

# Or use defined date range
# startdate="0601"
# enddate="0930"
# dates=[]
# for year in range(2019,2020):
#     print(year)
#     start = str(year)+startdate
#     end = str(year)+enddate
#     dates.extend(pd.date_range(start=start,end=end).strftime("%Y-%m-%d").to_list())

In [ ]:
url_dataspace = "https://catalogue.dataspace.copernicus.eu/odata/v1"
import time

# Filtering
satellite = "SENTINEL-3"
level= "LST"
cloud_cover_max = 0.2

# Set your username and password here
username = "your_username"
password = "your_password"

# Area of Interest (AOI) - Define a bounding box or polygon for your area of interest
aoi_polygon = "POLYGON ((-74.255591 40.496134,-74.255591 40.915533,-73.700009 40.915533,-73.700009 40.496134,-74.255591 40.496134))"

# Download folder
folder_path = "LST zips"
os.makedirs(folder_path, exist_ok=True)

product_id_df = pd.DataFrame()
for start_date in dates:
  end_date = start_date
  start_date_full =start_date+"T00:00:00.000Z"
  end_date_full = end_date +"T23:59:59.000Z"

  query = f"{url_dataspace}/Products?$filter=Collection/Name eq '{satellite}' \
        and Attributes/OData.CSC.StringAttribute/any(att:att/Name eq 'productType' \
        and att/OData.CSC.StringAttribute/Value eq '{level}') \
        and OData.CSC.Intersects(area=geography'SRID=4326;{aoi_polygon}') \
        and ContentDate/Start gt {start_date_full} \
        and ContentDate/Start lt {end_date_full}"

  response = requests.get(query).json()
  result = pd.DataFrame.from_dict(response["value"])
  time.sleep(3)


  if result.empty:
    continue
  
  filtered_df = result[(result["Name"].str.split("_").str[8].str.contains("T12|T13|T14|T15|T16|T17"))&(result["Name"].str.contains("NT"))]

  filtered_df = filtered_df.reset_index(drop=True)

  product_id_df = pd.concat([product_id_df, filtered_df], ignore_index=True)
auth_server_url = "https://identity.dataspace.copernicus.eu/auth/realms/CDSE/protocol/openid-connect/token"
data = {
    "client_id": "cdse-public",
    "grant_type": "password",
    "username": username,
    "password": password,
}


products_dict = [
    {
        "product_id": row["Id"],
        "product_name": row["Name"]
    }
    for _, row in product_id_df.iterrows()
]
products_dict


In [ ]:
for product in products_dict:
    product_id = product["product_id"]
    product_name = product["product_name"]
    file_path = os.path.join(folder_path, f"{product_name}.zip")
    
    # Skip download if file already exists
    if os.path.exists(file_path):
        print(f"File {file_path} already exists. Skipping download.")
        continue

    # Get authentication token
    response = requests.post(auth_server_url, data=data, verify=True, allow_redirects=False)
    access_token = json.loads(response.text)["access_token"]
    
    # Configure download URL and headers
    download_url = f"https://zipper.dataspace.copernicus.eu/odata/v1/Products({product_id})/$value"
    headers = {"Authorization": f"Bearer {access_token}"}

    # Initialize session for download
    with requests.Session() as session:
        session.headers.update(headers)

        try:
            # Start download stream
            response = session.get(download_url, stream=True, allow_redirects=False)

            # Retry after delay if request fails
            if response.status_code != 200:
                print(f"Download failed with status code: {response.status_code}. Retrying in 2 minutes.")
                time.sleep(120)
                continue

            # Save file in chunks
            with open(file_path, "wb") as file:
                for chunk in response.iter_content(chunk_size=8192):
                    if chunk:
                        file.write(chunk)

        except requests.exceptions.RequestException as e:
            print(f"Request error: {e}. Retrying in 80 seconds.")
            time.sleep(80)